In [8]:
!ffmpeg -i /home/cocolng/Retrieval-based-Voice-Conversion-WebUI/opt/Macron_Discours_Europe_1h30_main_vocal.wavd -ac 1 -f segment -segment_time 600 -c:a pcm_s16le /home/cocolng/Retrieval-based-Voice-Conversion-WebUI/opt/Macron/segment_%03d.wav

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [1]:
!nvidia-smi

Mon Dec  2 12:56:30 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A6000               Off |   00000000:A1:00.0 Off |                  Off |
| 30%   24C    P8             21W /  300W |       2MiB /  49140MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import subprocess
import threading
import time
import sys
import os
import signal
import requests
import psutil
import torch
import atexit
import warnings
import tensorflow
from datetime import datetime

# Nettoyer la mémoire GPU au démarrage si CUDA est disponible
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(f"\nGPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM Totale: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB") 
    
def cleanup_semaphores():
    """Nettoie les sémaphores au shutdown"""
    try:
        # Récupérer le PID du processus actuel
        current_pid = os.getpid()
        
        # Trouver et nettoyer les sémaphores associés à ce processus
        for sem in psutil.Process(current_pid).children(recursive=True):
            try:
                sem.terminate()
                sem.wait(timeout=3)
            except (psutil.NoSuchProcess, psutil.TimeoutExpired):
                pass
                
        # Force le nettoyage du resource tracker
        if hasattr(resource_tracker, '_resource_tracker'):
            resource_tracker._resource_tracker._stop = True
            
    except Exception as e:
        print(f"Erreur lors du nettoyage des sémaphores: {e}")

class GradioNgrokLauncher:
    def __init__(self):
        self.gradio_process = None
        self.ngrok_process = None
        self.is_running = False
        self.stdout_thread = None
        self.stderr_thread = None
        atexit.register(cleanup_semaphores)

    def kill_process_tree(self, pid):
        """Tue un processus et tous ses enfants"""
        try:
            parent = psutil.Process(pid)
            children = parent.children(recursive=True)
            
            # Tuer les processus enfants
            for child in children:
                try:
                    child.kill()
                except psutil.NoSuchProcess:
                    pass
            
            # Tuer le processus parent
            parent.kill()
            
            # Attendre que tout soit terminé
            psutil.wait_procs(children + [parent], timeout=5)
        except psutil.NoSuchProcess:
            pass

    def start_ngrok(self):
        """Démarre le tunnel ngrok"""
        try:
            self.ngrok_process = subprocess.Popen(
                ['ngrok', 'http', '7865'],
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE
            )
            time.sleep(3)
            
            try:
                response = requests.get('http://localhost:4040/api/tunnels', timeout=5)
                tunnels = response.json()['tunnels']
                if tunnels:
                    public_url = tunnels[0]['public_url']
                    print(f"\n🌐 URL PUBLIQUE NGROK: {public_url}\n")
            except Exception as e:
                print(f"Erreur lors de la récupération de l'URL ngrok: {e}")
        except Exception as e:
            print(f"Erreur lors du démarrage de ngrok: {e}")

    def stream_output(self, pipe, prefix=''):
        """Capture la sortie du processus Gradio"""
        while self.is_running:
            try:
                line = pipe.readline()
                if not line and not self.is_running:
                    break
                if line:
                    decoded_line = line.decode().strip()
                    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    print(f"[{timestamp}] {prefix}{decoded_line}")
                    sys.stdout.flush()
            except:
                break

    def handle_shutdown(self, signum=None, frame=None):
        """Gestion de l'arrêt"""
        print("\n=== Arrêt demandé ===\n")
        self.is_running = False
        self.stop()

    def start(self):
        """Démarre Gradio et ngrok"""
        print("\n=== Démarrage de l'application avec tunnel ngrok ===\n")
        self.is_running = True

        # Capture des signaux d'arrêt
        signal.signal(signal.SIGINT, self.handle_shutdown)
        signal.signal(signal.SIGTERM, self.handle_shutdown)

        try:
            self.start_ngrok()
            
            self.gradio_process = subprocess.Popen(
                ['python', '-u', 'infer-web.py'],
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                env=dict(os.environ, PYTHONUNBUFFERED='1')
            )

            self.stdout_thread = threading.Thread(
                target=self.stream_output,
                args=(self.gradio_process.stdout, '')
            )
            self.stderr_thread = threading.Thread(
                target=self.stream_output,
                args=(self.gradio_process.stderr, 'ERR: ')
            )
            
            self.stdout_thread.daemon = True
            self.stderr_thread.daemon = True
            self.stdout_thread.start()
            self.stderr_thread.start()

            while self.is_running:
                if self.gradio_process.poll() is not None:
                    self.is_running = False
                    print("\n⚠️ Le processus Gradio s'est arrêté!\n")
                    break
                time.sleep(0.1)

        except Exception as e:
            print(f"\n⚠️ Erreur: {e}\n")
        finally:
            print("Pour etre sur de l'arret, on re-try le stop")
            self.handle_shutdown()

    def stop(self):
        """Arrête tous les processus"""
        # Arrêt des processus Gradio
        cleanup_semaphores() 
        if self.gradio_process:
            try:
                self.kill_process_tree(self.gradio_process.pid)
                print("✓ Gradio arrêté")
            except Exception as e:
                print(f"⚠️ Erreur lors de l'arrêt de Gradio: {e}")

        # Arrêt des processus ngrok
        if self.ngrok_process:
            try:
                self.kill_process_tree(self.ngrok_process.pid)
                print("✓ Ngrok arrêté")
            except Exception as e:
                print(f"⚠️ Erreur lors de l'arrêt de ngrok: {e}")

        # Attente des threads avec timeout
        if self.stdout_thread:
            self.stdout_thread.join(timeout=2)
        if self.stderr_thread:
            self.stderr_thread.join(timeout=2)

if __name__ == "__main__":
    warnings.filterwarnings("ignore", 
                          category=UserWarning, 
                          module="multiprocessing.resource_tracker")
    launcher = GradioNgrokLauncher()
    launcher.start()

2025-01-22 15:08:36.597829: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737554916.635825  433173 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737554916.648985  433173 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-22 15:08:36.760269: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.



GPU: NVIDIA RTX A6000
VRAM Totale: 50.93 GB

=== Démarrage de l'application avec tunnel ngrok ===


🌐 URL PUBLIQUE NGROK: https://cdb3-134-157-107-171.ngrok-free.app

[2025-01-22 15:08:46] ERR: 2025-01-22 15:08:46.726751: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
[2025-01-22 15:08:46] ERR: WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
[2025-01-22 15:08:46] ERR: E0000 00:00:1737554926.742251  433309 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
[2025-01-22 15:08:46] ERR: E0000 00:00:1737554926.746978  433309 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[2025-01-22 15:08:46] ERR: 2025-01-22 15:08:46.766505: I tensorflow/core/p

In [ ]:
# Dans infer/modules/train/train.py
config = {
    "train": {
        "batch_size": 27, 
        "gpu_worker": 48,
        "cache_all_data": False, "
        "epochs": 800,
        "save_every_epoch": 50
    }
}

In [2]:
!pip install protobuf==3.20.1

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.3
    Uninstalling protobuf-5.29.3:
      Successfully uninstalled protobuf-5.29.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-core 2.24.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0.dev0,>=3.19.5, but you have protobuf 3.20.1 which is incompatible.
googleapis-common-protos 1.66.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0.dev0,>=3.20.2, but you have protobuf 3.20.1 which is incompatible.
tensorflow 2.18.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 3.20